In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from pathlib import Path

c:\Users\Mi\Документы\GitHub\AITH-ML-Python\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_PATH = "C:/Users/Mi/Документы/GitHub/AITH-ML-Python/models_sources/gpt_model"

In [13]:
class TextGenerator:
    def __init__(self, model_name='fine_tuned_model', data_path=DATA_PATH):
        """
        Инициализация модели и токенизатора.
        Загружаем модель и токенизатор из указанного пути.
        """
        model_path = Path(data_path) / model_name
        self.tokenizer = GPT2Tokenizer.from_pretrained(str(model_path))
        self.model = GPT2LMHeadModel.from_pretrained(str(model_path))
        self.model.eval()

    def generate_text(
            self, 
            prompt: str, 
            max_length=100,
            num_return_sequences=1,
            temperature=1.0, 
            top_k=0, 
            top_p=1.0, 
            do_sample=False,
            ):
        """
        Генерация текста на основе заданного начального текста (prompt) и параметров.
        
        Параметры:
        - prompt: Текст пользователя.
        - max_length: Максимальная длина сгенерированного текста.
        - num_return_sequences: Количество возвращаемых последовательностей.
        - temperature: Контролирует разнообразие вывода.
        - top_k: Если больше 0, ограничивает количество слов для выборки только k наиболее вероятными словами.
        - top_p: Если меньше 1.0, применяется nucleus sampling.
        - do_sample: Если True, включает случайную выборку для увеличения разнообразия.
        """
        encoded_input = self.tokenizer.encode(prompt, return_tensors='pt')
        prompt_length_in_tokens = self.count_tokens(prompt)

        generator = pipeline("text-generation", model=self.model, tokenizer=self.tokenizer, truncation=True)
        all_texts = generator(
            prompt,
            max_length=max_length + prompt_length_in_tokens,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            do_sample=do_sample,
            top_k=top_k,
            top_p=top_p
            )
        
        # prompt_length = len(self.tokenizer.decode(encoded_input[0], skip_special_tokens=True))
        prompt_length_in_chars = self.count_chars(prompt)
        all_texts = [text["generated_text"] for text in all_texts]
        # trimmed_texts = [text[prompt_length:] for text in all_texts]
        trimmed_texts = [text[prompt_length_in_chars:] for text in all_texts]
        
        return {
            "full_texts": all_texts,
            "generated_texts": trimmed_texts
        }
    
    def count_tokens(self, input: str) -> int:
        return len(self.tokenizer.encode(input, return_tensors="pt")[0])
    
    def count_chars(self, input: str) -> int:
        return len(input)

In [14]:
generator = TextGenerator(
    model_name='',
    data_path=DATA_PATH
)

In [15]:
prompt = "Мой дядя самых честных правил"

generated_texts = generator.generate_text(
    prompt=prompt,
    max_length=50,
    num_return_sequences=2,
    do_sample=True,
    temperature=0.95,
    top_k=10,
    top_p=0.95
)

print(generated_texts)

for i, text in enumerate(generated_texts['full_texts']):
    print(f"\nGenerated Text {i+1}:\n{text}")

Device set to use cpu


{'full_texts': ['Мой дядя самых честных правил\nЗасеребрился, а стал таким скучным.\nОн пьет чай, ест котлету,\nВ углу книжки читает;\nНа всех двух дочек глядит\nСтарик, седой, в пенсне.\nОн пьет чай', 'Мой дядя самых честных правил,\nОн знал толк во всем и в том,\nЧто такое порядочный человек;\nЗа ним все были друзья и друзья». —\n«А ты, мой Евгений,\nКакое благородное дело?\nВ чем же твое благородство'], 'generated_texts': ['\nЗасеребрился, а стал таким скучным.\nОн пьет чай, ест котлету,\nВ углу книжки читает;\nНа всех двух дочек глядит\nСтарик, седой, в пенсне.\nОн пьет чай', ',\nОн знал толк во всем и в том,\nЧто такое порядочный человек;\nЗа ним все были друзья и друзья». —\n«А ты, мой Евгений,\nКакое благородное дело?\nВ чем же твое благородство']}

Generated Text 1:
Мой дядя самых честных правил
Засеребрился, а стал таким скучным.
Он пьет чай, ест котлету,
В углу книжки читает;
На всех двух дочек глядит
Старик, седой, в пенсне.
Он пьет чай

Generated Text 2:
Мой дядя самых чест